In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
names = ['HUC_02','GAGE_ID','GAGE_NAME','LATITUDE','LONGITUDE','ELEVATION']
gauges = pd.read_table('/Volumes/data/basin_dataset_public/basin_metadata/gauge_information.txt',usecols=[0,1,2,3,4,5],names=names,skiprows=1)

In [3]:
basins = pd.read_table('/Volumes/data/basin_dataset_public/basin_metadata/basin_physical_characteristics.txt',delim_whitespace=True,skiprows=0)

In [4]:
import glob

In [5]:
streamfiles = glob.glob('/Volumes/data/basin_dataset_public/usgs_streamflow/*/*.txt') # grab all the streamflow records

## Process Streamflow records

In [6]:
# Create Data Frames
data = pd.DataFrame(columns=['year','Q','gage']) # this will be Q by watershed and year
data2 = pd.DataFrame(columns=['Q','gage']) # this will be the total Q for each watershed

# define the header
header=['basin','year','month','day','Q','Qual']

# loop through each streamflow file
for fl in streamfiles:
    gage = int(fl.split('/')[-1].split('.')[0].split('_')[0]) # grab the gage ID from the file name
    dat = pd.read_table(fl,delim_whitespace=True,names=header) # read in the file
    
    dat.Q*=86400. # multiply by the number of seconds in the day to get the total volume [CF]
    dat.Q *= 0.0283168 #convert CF to cubic meters [m3]
    
    dat2 = dat.groupby(by='year').sum() # generate yearly sums
    
    dat2.reset_index(inplace=True) # this pulls the years out of the index to the column
    n = len(dat2) # get the length of the annual data
    dat2['gage'] = np.repeat(gage,n) # create a column of the gage ID
    data = data.append(dat2[['year','Q','gage']]) # Save the yearly data to the yearly data frame
    
    # save the summed data for this watershed
    dat3 = pd.DataFrame({'Q':dat.Q.sum(),'gage':gage},index=[0]) # generate long term Q sum in m3
    data2 = data2.append(dat3) # save the cumulative data to the cumulative data frame

## Convert Streamflow Volumes to absolute discharge

In [7]:
missing = [] # make a missing data list
for gage in np.unique(data.gage):
    if basins.loc[basins.BASIN_ID==gage].empty:
        missing.append(gage) # if a basin is missing, add it to this list
    else: # otherwise
        area = basins.loc[basins.BASIN_ID==gage,'Size(km2)'].as_matrix() # pull the basin area from the basins data frame
        area *= 1000000. # convert from km2 to m2
        
        # change the yearly data
        data.loc[data.gage==gage,'Q'] /= area # convert the discharge from m3 to m
        data.loc[data.gage==gage,'Q'] *= 1000. # convert m to mm
        
        # change the cumulative data
        data2.loc[data2.gage==gage,'Q'] /= area # convert the discharge from m3 to m
        data2.loc[data2.gage==gage,'Q'] *= 1000. # convert m to mm
        

## Remove missing records from the dataframe

In [8]:
for gage in missing:
    data.loc[data.gage==gage,:] = np.NaN
    data2.loc[data2.gage==gage,:] = np.NaN

In [9]:
data.dropna(inplace=True)
data2.dropna(inplace=True)

In [10]:
data.to_pickle('../data/processed_observed_streamflow.pcl')
data2.to_pickle('../data/processed_observed_streamflow_sums.pcl')